# 토큰 계산하기

In [ ]:
import tiktoken
import json

def pick_encoding(model: str):
    """
    모델명에 따라 적절한 토크나이저(encoding)를 선택합니다.
    GPT-4o / GPT-4.1 / o1~o4 계열은 o200k_base, 
    그 외 (GPT-3.5 등)은 cl100k_base 사용.
    """
    if any(m in model for m in ["gpt-4o", "gpt-4.1", "o4", "o3", "o1"]):
        return tiktoken.get_encoding("o200k_base")
    return tiktoken.get_encoding("cl100k_base")


def count_input_tokens(messages, model="gpt-4o-mini"):
    """
    메시지 목록의 총 토큰 수를 계산합니다.
    messages: [{"role":"system","content":"..."}, {"role":"user","content":"..."}]
    """
    enc = pick_encoding(model)
    # 실제 전송 구조에 근사하도록 payload를 구성 (input 필드 사용)
    payload = {"model": model, "input": messages}
    text = json.dumps(payload, ensure_ascii=False, separators=(",", ":"))
    return len(enc.encode(text))


# ✅ 사용 예시
if __name__ == "__main__":
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕! 오늘 날씨 어때?"}
    ]

    tokens = count_input_tokens(messages, model="gpt-4o-mini")
    print(f"예상 입력 토큰 수: {tokens}")

In [ ]:
import tiktoken, json

# ① 모델별 단가 (2025년 10월 기준, USD)
PRICE_TABLE = {
    "gpt-4o-mini": {"input": 0.000150, "output": 0.000600},   # per 1K tokens
    "gpt-4o": {"input": 0.0050, "output": 0.0150},
    "gpt-3.5-turbo": {"input": 0.0005, "output": 0.0015},
}

# ② 모델별 토크나이저 선택
def pick_encoding(model: str):
    if any(m in model for m in ["gpt-4o", "gpt-4.1", "o4", "o3", "o1"]):
        return tiktoken.get_encoding("o200k_base")
    return tiktoken.get_encoding("cl100k_base")

# ③ 입력 메시지 토큰 계산
def count_input_tokens(messages, model="gpt-4o-mini"):
    enc = pick_encoding(model)
    payload = {"model": model, "input": messages}
    text = json.dumps(payload, ensure_ascii=False, separators=(",", ":"))
    return len(enc.encode(text))

# ④ 예상 비용 계산
def estimate_cost(input_tokens, output_tokens, model="gpt-4o-mini"):
    prices = PRICE_TABLE.get(model)
    if not prices:
        raise ValueError(f"지원되지 않는 모델명: {model}")
    cost = (input_tokens / 1000) * prices["input"] + (output_tokens / 1000) * prices["output"]
    return round(cost, 6)  # USD 단위, 소수점 6자리까지 표시

# ⑤ 실행 예시
if __name__ == "__main__":
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "오늘 서울 날씨 어때?"}
    ]

    model = "gpt-4o-mini"

    # (1) 입력 토큰 계산
    input_tokens = count_input_tokens(messages, model=model)

    # (2) 출력 토큰은 예시로 200개라고 가정
    output_tokens = 200

    # (3) 비용 계산
    cost_usd = estimate_cost(input_tokens, output_tokens, model=model)

    print(f"입력 토큰 수: {input_tokens}")
    print(f"출력 토큰 수: {output_tokens}")
    print(f"예상 비용: ${cost_usd} USD (약 {cost_usd*1400:.2f}원 기준)")